# Training

モデルをトレーニングするには３つのことが必要だ。
* 目的関数、いくつかの入力データが与えられたらモデルがどれだけよく機能するかを評価
* 目的関数に提供されるデータポイントのコレクション
* 適切にモデルパラメータを更新する[オプティマイザ](https://fluxml.ai/Flux.jl/stable/training/optimisers.html)

これらを`Flux.train!`で呼ぶことができる。

```julia
Flux.train!(objective, data, opt)
```

[model zoo](https://github.com/FluxML/model-zoo)に沢山の例がある。

## Loss Functions
---

目的関数はモデルの損失を返す必要がある。  
basicsで定義した`loss`関数は、目的として動作する。  
あるモデルに関して目的を定義することもできる。

In [10]:
using Flux
m = Chain(
    Dense(28^2, 32, σ),
    Dense(32,10), softmax)

loss(x,y) = Flux.mse(m(x), y)

# 元コードにはないが必要であろうものを追記
opt = SGD(params(m),0.1)
# 追記終わり

# later
Flux.train!(loss, data, opt)

ほとんどの目的はターゲット`y`から予測`m(x)`の距離の指標である、あるコスト関数で定義される。  
Fluxでは、平均二条誤差`mse`やクロスエントロピー損失の`crossentropy`のような、いくつかが組み込まれているが、必要に応じて計算できる。 

## Detasets
---

`data`引数は訓練のためのデータのコレクションを提供する（普通は入力`x`とターゲット出力`y`のセット）。  
例えば、ここにデータポイントが一つだけのダミーデータセットがある。 

In [3]:
x = rand(28^2)
y = rand(10)
data = [(x,y)]

1-element Array{Tuple{Array{Float64,1},Array{Float64,1}},1}:
 ([0.627661, 0.846875, 0.321991, 0.507268, 0.628431, 0.499641, 0.170108, 0.0573684, 0.582821, 0.07334  …  0.51649, 0.563545, 0.290845, 0.66132, 0.584451, 0.591893, 0.696098, 0.0442405, 0.349206, 0.979701], [0.0320805, 0.89024, 0.629555, 0.64725, 0.202011, 0.812543, 0.0884321, 0.0193722, 0.744511, 0.998158])

`Flux.train!`は`loss(x,y)`を呼び、勾配計算をして、重みを更新して、次のデータがあればそれに移動する。  
三回、同じデータでモデルを訓練できる.

In [4]:
data = [(x,y),(x,y),(x,y)]
# または以下の書き方でも同様
data = Iterators.repeated((x,y),3)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,1},Array{Float64,1}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,1},Array{Float64,1}}}(([0.627661, 0.846875, 0.321991, 0.507268, 0.628431, 0.499641, 0.170108, 0.0573684, 0.582821, 0.07334  …  0.51649, 0.563545, 0.290845, 0.66132, 0.584451, 0.591893, 0.696098, 0.0442405, 0.349206, 0.979701], [0.0320805, 0.89024, 0.629555, 0.64725, 0.202011, 0.812543, 0.0884321, 0.0193722, 0.744511, 0.998158])), 3)

`xs`と`ys`は別でロードするのが普通。  
このケースでは`zip`を使うことができる。

In [6]:
using Flux: @epochs

@epochs 2 println("hello")

hello
hello


┌ Info: Epoch 1
└ @ Main /Users/umikoz/.julia/packages/Flux/jsf3Y/src/optimise/train.jl:93
┌ Info: Epoch 2
└ @ Main /Users/umikoz/.julia/packages/Flux/jsf3Y/src/optimise/train.jl:93


In [16]:
@epochs 2 Flux.train!(loss, data, opt)
# 2エポック学習する

┌ Info: Epoch 1
└ @ Main /Users/umikoz/.julia/packages/Flux/jsf3Y/src/optimise/train.jl:93
┌ Info: Epoch 2
└ @ Main /Users/umikoz/.julia/packages/Flux/jsf3Y/src/optimise/train.jl:93


## Callbacks
---

`train!`は追加の引数を取り、`cb`、これはコールバックに使われるので、トレーニングプロセスを監視できる。  
例えば、

In [31]:
using Flux:train!
train!(loss, data, opt; cb= ()-> println("training"))

training
training
training


コールバックは教師データのバッチごとに呼び出される。  
`Flux.throttle(f, timeout)`を使ってこれを遅くすることができる。これは`timeout`秒の間に1回以上`f`が呼ばれるのを防ぐ。

もっと典型的には次のようになる。

In [41]:
using Flux: throttle

test_x, test_y = x, y# テストデータの1バッチを作る
evalcb() = @show(loss(test_x, test_y))

Flux.train!(loss, data, opt, cb = throttle(evalcb, 5))

loss(test_x, test_y) = 0.25721523745290814 (tracked)


In [42]:
# ここちょっとよく分かってない